# Amazon Bedrock Guardrails による Strands エージェントの保護

## プロンプトの言語について
このワークショップの環境は、**Content filters tier** と **Denied topics tier** が **Classic** の前提のため、プロンプトは英語のままにしています。

## 概要
この例では、[Amazon Bedrock Guardrails](https://aws.amazon.com/bedrock/guardrails/) を使用して最初の Strands エージェントを作成し、基盤となる Amazon Bedrock LLM モデルを保護する方法について説明します。

Amazon Bedrock Guardrails は、大規模な生成 AI アプリケーションを安全に構築するための設定可能なセーフガードを提供します。Amazon Bedrock でサポートされている FM、微調整されたモデル、Amazon Bedrock 外でホストされているモデルなど、幅広い基盤モデル (FM) に一貫した標準的なアプローチを適用する Guardrails は、業界をリードする安全性保護を提供します。

Strands エージェントを使用すると、Amazon Bedrock モデルに Amazon Bedrock Guardrails を直接追加できます。 Amazon Bedrock モデルを使用していない場合は、[Apply Guardrail API](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-use-independent-api.html) を使用してモデルを保護できます。この場合、先ほど学習した高度な処理機能を使用してパイプラインを構築する必要があります。

## エージェントの詳細
<div style="float: left; margin-right: 20px;">

|機能 |説明 |
|--------------------|---------------------------------------------------|
|使用されている AWS サービス |Amazon Bedrock Guardrails |
|作成されたカスタムツール |get_customer_profile、list_customer_purchases、list_customer_tickets、update_customer_profile|
|エージェントの構造 |単一エージェントアーキテクチャ |

</div>


## アーキテクチャ

<div style="text-align:left">
<img src="images/architecture.png" width="85%" />
</div>

## 主な機能
* **単一エージェントアーキテクチャ**: この例では、組み込みツールとカスタムツールと連携する単一のエージェントを作成します。
* **カスタムツール**: 独自のツールを作成する方法を学びます。
* **基盤となる LLM として Amazon Bedrock モデルを使用**: 基盤となる LLM モデルとして Amazon Bedrock の Anthropic Claude 3.7 を使用しました。
* **Amazon Bedrock Guardails を基盤となる安全対策として使用**: Amazon Bedrock Guardrails を使用してエージェントアプリケーションを保護します。

## セットアップと前提条件

### 前提条件
* Python 3.10 以上
* AWS アカウント
* Amazon Bedrock で有効化された Anthropic Claude 3.7
* Amazon Bedrock Guardrails を作成する権限を持つ IAM ロール

Strands エージェントに必要なパッケージをインストールしましょう

In [ ]:
!pip install -U -r requirements.txt

### 依存パッケージのインポート

依存パッケージをインポートしましょう

In [ ]:
import json
import boto3
import os
from strands import Agent, tool
from strands.models import BedrockModel
from customer_profile_tools import get_customer_profile, list_customer_purchases, list_customer_tickets, update_customer_profile
from customer_profiles import CustomerProfileManager, generate_synthetic_profiles

### データの生成

この例のために合成データを生成してみましょう。実際にはエージェントは既存の顧客データベースに接続しますが、この例ではJSONファイルを使用します。

In [ ]:
 # 必要に応じて合成プロファイルを生成
profile_manager = CustomerProfileManager()
if not os.path.exists("customer_profiles.json"):
    print("Generating synthetic customer profiles")
    profiles = generate_synthetic_profiles(10)
    print(f"Generated {len(profiles)} synthetic customer profiles")
else:
    print("Using existing customer profiles")
    profiles = list(profile_manager.profiles.values())
    print(f"Loaded {len(profiles)} customer profiles")

# 参考のためにサンプルプロファイルを表示
if profiles:
    sample_profile = profiles[0]
    print(f"\nSample Profile: {sample_profile.name}")
    print(f"Email: {sample_profile.email}")
    print(f"Customer ID: {sample_profile.customer_id}")
    print(f"Location: {sample_profile.state}, {sample_profile.country}")
    print(f"Purchases: {len(sample_profile.purchase_history)}")
    print(f"Support Tickets: {len(sample_profile.support_tickets)}")

## Amazon Bedrock ガードレールの作成

では、顧客向けアプリケーションにおける金融アドバイスの提供を防止し、よくある課題をフィルタリングするための Amazon Bedrock ガードレールを作成しましょう。

ガードレールを作成するには、まず Amazon Bedrock クライアントを設定する必要があります。

In [ ]:
# AWSクライアントの構成
bedrock_client = boto3.client('bedrock')
bedrock_runtime = boto3.client('bedrock-runtime')

これで、`create_guardrail` メソッドを使用して、アプリケーションで使用する `no-investment-advice` ガードレールを作成できます。この例では、ガードレールに `Fiduciary Advice` のトピック拒否を追加し、エージェントによる受託者へのアドバイスをブロックします。ガードレールには、不適切なコンテンツをフィルタリングするための基本的なコンテンツポリシーと、特定の定義済み単語を検出するための単語ポリシー設定も含まれます。

In [ ]:
# Bedrock Guardrail の作成
response = bedrock_client.create_guardrail(
    name='no-investment-advice',
    description='Prevents the model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {
                'type': 'PROFANITY'
            }
        ]
    },
    blockedInputMessaging='I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including any financial, or restricted details.',
    blockedOutputsMessaging='I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including financial, or restricted details.',
)

# 応答を印刷してガードレール ID を取得
print("Guardrail ID:", response.get('guardrailId'))
print("Guardrail ARN:", response.get('guardrailArn'))

# 後で使用するためにガードレール ID を保存


In [ ]:
guardrail_id = response.get('guardrailId')
guardrail_version = "DRAFT"  #初期バージョンは常に 1 

### ガードレールを直接テストする

ガードレールが期待通りに動作することを確認するために、`test_guardrail` サポート関数を作成します。この関数は `apply_guardrail` メソッドを使用して入力テキストを保護し、ガードレールによって実行されるアクションに関する情報を提供します。

In [ ]:
# ガードレールによって入出力がブロックされているかどうかを確認するテスト機能
def test_guardrail(text, source_type='INPUT'):
      response = bedrock_runtime.apply_guardrail(
          guardrailIdentifier=guardrail_id,
          guardrailVersion=guardrail_version,
          source=source_type,  # can be 'INPUT' or 'OUTPUT'
          content=[{"text": {"text": text}}]
      )

      # 新しい応答形式では異なるフィールドが使用される
      print(f"Action: {response.get('action')}")
      print(f"Action Reason: {response.get('actionReason', 'None')}")

      # コンテンツがブロックされているかどうかを確認
      is_blocked = response.get('action') == 'GUARDRAIL_INTERVENED'
      print(f"Content {source_type} blocked: {is_blocked}")

      if is_blocked:
          # トリガーされたトピックポリシーを印刷
          assessments = response.get('assessments', [])
          if assessments and 'topicPolicy' in assessments[0]:
              print("Blocked topics:", [topic.get('name') for topic in
  assessments[0]['topicPolicy'].get('topics', [])
                                       if topic.get('action') == 'BLOCKED'])

          # 可能な場合は変更された出力を印刷
          if 'outputs' in response and response['outputs']:
              print("Modified content:", response['outputs'][0].get('text', 'None'))

      return response

# 安全な入力をテスト
print("Testing safe input:")
test_guardrail("Tell me about general financial literacy concepts.")

# ブロックする必要があるテスト入力
print("\nTesting input that should be blocked:")
test_guardrail("What stocks should I invest in for my retirement?")

## Strands エージェントとの統合

ガードレールが期待どおりに動作していることを確認したので、Amazon Bedrock ガードレールと Strands エージェントを統合してみましょう。これは、Bedrock モデルオブジェクトで `guardrail_id`、`guardrail_version`、`guardrail_trace` を設定することで行います。モデルオブジェクトを作成したら、それを使用してエージェントを作成できます。このエージェントでは、`get_customer_profile`、`list_customer_purchases`、`list_customer_tickets`、`update_customer_profile` というカスタムツールを使用します。これらの実装を確認するには、`customer_profile_tools.py` ファイルを確認してください。

In [ ]:
guardrail_id, guardrail_version

In [ ]:
# ガードレール構成の Bedrock モデルを作成
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version,
    # デバッグ用のトレースを有効化
    guardrail_trace="enabled"
)

# ガードレール保護モデルを使用してエージェントを作成
agent = Agent(
    system_prompt="You are a helpful assistant that provides customer support for retail products.",
    model=bedrock_model,
    tools=[
        get_customer_profile,
        list_customer_purchases,
        list_customer_tickets,
        update_customer_profile
    ]
)

### ガードレールを用いたStrandsエージェントのテスト

安全な入力と危険な入力の両方でエージェントをテストしてみましょう。そのためには、エージェントの応答を処理し、`stop_reason`がガードレール介入によるものかどうかを確認します。

In [ ]:
# エージェントをテストし、ガードレール介入をチェックするヘルパー関数
def test_agent_with_guardrail(prompt):
    print(f"\nUser: {prompt}")

    # エージェントの応答を取得
    response = agent(prompt)

    # ガードレール介入の確認
    if hasattr(response, 'stop_reason') and response.stop_reason == "guardrail_intervened":
        print("\n ⚠️ GUARDRAIL INTERVENED!")
        #print(f"Response: {response}")
    else:
        return response

In [ ]:
# 安全な質問でテスト
test_agent_with_guardrail(
    " what is my latest purchase my Customer ID is CUST100?"
)

In [ ]:
# 投資の推奨、憎悪や暴力について尋ねる質問でテスト
test_agent_with_guardrail(
    "My SSN is 123-45-6789. Can you help me understand how to protect my retirement account?"
)

## 会話履歴の調査

ガードレールがどのように影響したかを確認するために、会話履歴を調べてみましょう。

In [ ]:
# 会話履歴を出力
print(f"Conversation history: {json.dumps(agent.messages, indent=4)}")

## チャットボットへの Bedrock ガードレールの統合

Amazon Bedrock には、Strands Agents SDK と直接統合できる組み込みのガードレールフレームワークが用意されています。ガードレールがトリガーされると、Strands Agents SDK は会話履歴内のユーザー入力を自動的に上書きします。これは、同じ質問によってフォローアップの質問がブロックされないようにするためです。これは、guardrail_redact_input ブール値と、guardrail_redact_input_message 文字列を使用して上書きメッセージを変更することで設定できます。また、モデルの出力にも同じ機能が組み込まれていますが、デフォルトでは無効になっています。guardrail_redact_output ブール値を使用してこれを有効にし、guardrail_redact_output_message 文字列を使用して上書きメッセージを変更できます。以下は、コードで Bedrock ガードレールを活用する例です。

In [ ]:
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    guardrail_id=guardrail_id,         # BedrockガードレールID
    guardrail_version=guardrail_version,             # ガードレールバージョン
    guardrail_trace="enabled",
    guardrail_redact_output = True,        
    guardrail_redact_input = True  ,
    guardrail_redact_input_message = "Guardrail Intervened and Redacted"     
)

# ガードレール保護モデルを使用してエージェントを作成
agent = Agent(
    system_prompt="You are a helpful assistant that provides customer support for retail products.",
    model=bedrock_model,
    tools=[
        get_customer_profile,
        list_customer_purchases,
        list_customer_tickets,
        update_customer_profile
    ]
)

In [ ]:
# カスタマーサポートエージェントに安全な質問をテストする
agent(f"what is the data from customer_id {sample_profile.customer_id}?")

In [ ]:
# グアドレールが介入し、入力がカスタムメッセージで編集されるユーザー入力によるテスト
agent("How should I allocate my 401(k) investments?")

編集された入力のカスタム メッセージは、次のように会話履歴に保存されます。

In [ ]:
print(f"Conversation history: {json.dumps(agent.messages, indent=4)}")

## クリーンアップ

ガードレールを以下のように削除してください。

In [ ]:
bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)

## おめでとうございます！

このノートブックでは、以下の方法を説明しました。

1. 金融アドバイスの提供を阻止する Amazon Bedrock ガードレールを作成する
2. Bedrock ランタイム API を使用してガードレールを直接テストする
3. ガードレールを Strands エージェントに統合する
4. さまざまな入力でエージェントをテストし、ガードレールの動作を確認する
5. Strands エージェントを使用して、チャットボットに Bedrock ガードレールを統合する
5. ガードレールを削除する

ガードレールは、AI の応答を安全かつコンプライアンスに準拠した状態に保ち、ユースケースに適した状態に保つのに役立ちます。
